# 1.1 Importando bibliotecas

In [2]:
import pandas as pd 
import numpy as np

# 1.2 Lendo a planilha

In [12]:
df = pd.read_csv("../../data/raw/DH_2022_Br_Regioes_20240423.xlsx - Sexo do Respontável.csv")

# 1.3 Visualizar primeiras linhas

In [13]:
df.head()

,N,Regiao,Sexo,Domicílios Rústicos,Domicílios Improvisados,Habitação Precária,Cômodos,Unidades conviventes déficit,Coabitação,Ônus,Déficit Habitacional (Habitação Precária + Coabitação + Ônus)
0,1,Norte,Homem,65.971,59.758,125.729,9.113,96.453,105.566,70.817,302.112
1,1,Norte,Mulher,51.022,151.997,203.019,7.809,144.027,151.836,113.848,468.702
2,1,Norte,Ignorado,0.000,2.514,2.514,0.000,0.000,0.000,0,2.514
3,2,Nordeste,Homem,163.408,127.456,290.864,4.706,125.825,130.530,231.049,652.444
4,2,Nordeste,Mulher,109.571,298.776,408.347,12.900,223.023,235.923,460.272,1.104.542


# 1.4 Renomear colunas

In [14]:
df.rename(columns={
    'Regiao' : 'regiao',
    'Sexo' : 'sexo',
    'Domicílios Rústicos' : 'domicilios_rusticos',
    'Domicílios Improvisados' : 'domicilios_improvisados',
    'Habitação Precária' : 'habitacao_precaria',
    'Cômodos' : 'comodos',
    'Unidades conviventes déficit' : 'unidades_conviventes_deficit',
    'Coabitação' : 'coabitacao',
    'Ônus' : 'onus',
    'Déficit Habitacional (Habitação Precária + Coabitação + Ônus)' : 'deficit_original'
}, inplace=True)

print(df)

    N        regiao      sexo  domicilios_rusticos  domicilios_improvisados  \
0   1         Norte     Homem               65.971                   59.758   
1   1         Norte    Mulher               51.022                  151.997   
2   1         Norte  Ignorado                0.000                    2.514   
3   2      Nordeste     Homem              163.408                  127.456   
4   2      Nordeste    Mulher              109.571                  298.776   
5   2      Nordeste  Ignorado                0.000                    4.045   
6   3       Sudeste     Homem               23.591                   76.908   
7   3       Sudeste    Mulher               28.327                  205.127   
8   3       Sudeste  Ignorado                0.000                    2.958   
9   4           Sul     Homem               68.097                   19.968   
10  4           Sul    Mulher               57.784                   46.330   
11  4           Sul  Ignorado                0.000  

# 1.5 Removendo ponto dos números e convertendo para inteiros

In [15]:
colunas_excluir = ["regiao", "sexo"]

colunas_converter = [col for col in df.columns if col not in colunas_excluir]

for col in colunas_converter:
    df[col] = (
        df[col]
        .astype(str).
        str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
    )
    df[col] = pd.to_numeric(df[col], errors='coerce')

print(df.dtypes)
df.head()

N                                int64
regiao                          object
sexo                            object
domicilios_rusticos              int64
domicilios_improvisados          int64
habitacao_precaria               int64
comodos                          int64
unidades_conviventes_deficit     int64
coabitacao                       int64
onus                             int64
deficit_original                 int64
dtype: object


,N,regiao,sexo,domicilios_rusticos,domicilios_improvisados,habitacao_precaria,comodos,unidades_conviventes_deficit,coabitacao,onus,deficit_original
0,1,Norte,Homem,65971,59758,125729,9113,96453,105566,70817,302112
1,1,Norte,Mulher,51022,151997,203019,7809,144027,151836,113848,468702
2,1,Norte,Ignorado,0,2514,2514,0,0,0,0,2514
3,2,Nordeste,Homem,163408,127456,290864,4706,125825,13053,231049,652444
4,2,Nordeste,Mulher,109571,298776,408347,129,223023,235923,460272,1104542


# 1.6 Validação e correção da coluna de déficit total

In [16]:
df['deficit_total'] = df['habitacao_precaria'] + df['coabitacao'] + df['onus']
df.drop(columns=['N', 'deficit_original'], inplace=True)

df.head()

,regiao,sexo,domicilios_rusticos,domicilios_improvisados,habitacao_precaria,comodos,unidades_conviventes_deficit,coabitacao,onus,deficit_total
0,Norte,Homem,65971,59758,125729,9113,96453,105566,70817,302112
1,Norte,Mulher,51022,151997,203019,7809,144027,151836,113848,468703
2,Norte,Ignorado,0,2514,2514,0,0,0,0,2514
3,Nordeste,Homem,163408,127456,290864,4706,125825,13053,231049,534966
4,Nordeste,Mulher,109571,298776,408347,129,223023,235923,460272,1104542


# 1.7 Separação dos dados Região vs. Total Brasil

In [17]:
df_regioes = df[df['regiao'] != 'Brasil'].copy()
df_brasil = df[df['regiao'] == 'Brasil'].copy()

print(df_regioes.head())
print(df_brasil.head())

     regiao      sexo  domicilios_rusticos  domicilios_improvisados  \
0     Norte     Homem                65971                    59758   
1     Norte    Mulher                51022                   151997   
2     Norte  Ignorado                    0                     2514   
3  Nordeste     Homem               163408                   127456   
4  Nordeste    Mulher               109571                   298776   

   habitacao_precaria  comodos  unidades_conviventes_deficit  coabitacao  \
0              125729     9113                         96453      105566   
1              203019     7809                        144027      151836   
2                2514        0                             0           0   
3              290864     4706                        125825       13053   
4              408347      129                        223023      235923   

     onus  deficit_total  
0   70817         302112  
1  113848         468703  
2       0           2514  
3  23104

# 1.8 Verificação dos dados

In [18]:
df_regioes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 0 to 14
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   regiao                        15 non-null     object
 1   sexo                          15 non-null     object
 2   domicilios_rusticos           15 non-null     int64 
 3   domicilios_improvisados       15 non-null     int64 
 4   habitacao_precaria            15 non-null     int64 
 5   comodos                       15 non-null     int64 
 6   unidades_conviventes_deficit  15 non-null     int64 
 7   coabitacao                    15 non-null     int64 
 8   onus                          15 non-null     int64 
 9   deficit_total                 15 non-null     int64 
dtypes: int64(8), object(2)
memory usage: 1.3+ KB
